<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">연습하기</font><br>
1. 본 코드를 보지 마시고, 처음 raw data 에서 사망자수 추이를 볼 수 있도록 데이터를 전처리해서 그래프 그리기 <br>
2. 본 코드를 보지 마시고, 처음 raw data 에서 회복자수 추이를 볼 수 있도록 데이터를 전처리해서 그래프 그리기    
</div> 

In [101]:
doc2 = pd.read_csv("COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/01-24-2021.csv")
doc2.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-25 05:22:16,33.93911,67.709953,54595,2378,47298,4919,Afghanistan,140.244867,4.355710
1,NaN,NaN,NaN,Albania,2021-01-25 05:22:16,41.15330,20.168300,72274,1315,43890,27069,Albania,2511.432344,1.819465
2,NaN,NaN,NaN,Algeria,2021-01-25 05:22:16,28.03390,1.659600,105596,2863,71945,30788,Algeria,240.806131,2.711277
3,NaN,NaN,NaN,Andorra,2021-01-25 05:22:16,42.50630,1.521800,9549,96,8652,801,Andorra,12358.765288,1.005341
4,NaN,NaN,NaN,Angola,2021-01-25 05:22:16,-11.20270,17.873900,19399,459,17266,1674,Angola,59.024043,2.366101


In [87]:
import pandas as pd
import json

with open("COVID-19-master/csse_covid_19_data/country_convert.json",'r') as json_file:
    json_data = json.load(json_file)

def country_name_convert(row):
    if row['Country_Region'] in json_data:
        row['Country_Region'] = json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')
    try:
        doc = doc[['Country_Region', 'Recovered']]
    except:
        doc = doc[['Country/Region', 'Recovered']]
        doc.columns = ['Country_Region', 'Recovered']
    doc = doc.dropna(subset=['Recovered'])
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)
    doc = doc.astype({'Recovered':'int64'})
    doc = doc.groupby('Country_Region').sum()

    data_column = filename.split(".")[0].lstrip('0').replace('-','/')
    doc.columns = [data_column]
    return doc

In [89]:
import os
from datetime import datetime

def generate_dateframe_by_path(PATH):
    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True
    for file in file_list:
        if file.split('.')[-1] == 'csv':
            csv_list.append(file)
    csv_list.sort(key=lambda x: datetime.strptime(x, '%m-%d-%Y.csv'))

    for file in csv_list:
        doc = create_dateframe(file)
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index = True, right_index = True)
    final_doc = final_doc.fillna(0)
    return final_doc

In [91]:
PATH = "COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/"
doc = generate_dateframe_by_path(PATH)

In [75]:
doc.to_csv("COVID-19-master/final_df_recovered.csv")

In [103]:
doc.head()

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,2/28/2023,3/01/2023,3/02/2023,3/03/2023,3/04/2023,3/05/2023,3/06/2023,3/07/2023,3/08/2023,3/09/2023
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
def create_flag_link(column):
    # 현재 국기 이미지를 가져올 수 있는 코드  
    flag_link = 'https://public.flourish.studio/country-flags/svg/' + column.lower() + '.svg'
    return flag_link


PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
df_confirmed = generate_dateframe_by_path(PATH)
df_confirmed = df_confirmed.astype('int64')

country_info = pd.read_csv("COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv", encoding='utf-8-sig', keep_default_na=False, na_values='')
country_info = country_info[['iso2', 'Country_Region']]
country_info = country_info.drop_duplicates(subset='Country_Region', keep='last')

doc_final_country = pd.merge(df_confirmed, country_info, how='left', on='Country_Region')
doc_final_country = doc_final_country.dropna(subset=['iso2'])
doc_final_country['iso2'] = doc_final_country['iso2'].apply(create_flag_link)

cols = doc_final_country.columns.tolist()
cols.remove('iso2')
cols.insert(1, 'iso2')
doc_final_country = doc_final_country[cols]
cols[1] = 'Country_Flag'
doc_final_country.columns = cols

doc_final_country.to_csv("COVID-19-master/final_covid_data_for_graph.csv")